<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/RAG_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. `RunnablePassthrouhg`**
---

Dentro de los Runnables, uno de los más sencillos —pero muy útil— es `RunnablePassthrough`. Este “pasa” sus datos de entrada directamente a la salida sin alterarlos. Puede parecer trivial, pero resulta práctico en situaciones en las que queremos que un eslabón de la cadena no modifique la información que recibe, sirviendo como “puente” para mantener la compatibilidad o facilidad de lectura en la cadena.

Su mayor valor está en mantener la estructura de Runnables y cadenas, permitiéndonos insertar fácilmente lógica futura o puntos de inspección.

# **Preparando el entorno del cuaderno**
---
Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).

- Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google Hugging Face, Mistral, Together, Anthropic, Deepseek

- Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

- Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.
El uso de las API de OpenAI y Anthropic es de pago. El resto son gratuitas y para usarlas basta con registrarse y generar una API Key.  

En el primer cuaderno encontraras los enlaces a estos servicios y este codigo explicado



In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')
DEEPSEEK_API_KEY=userdata.get('DEEPSEEK_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic


# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

## **Ejemplo 1: No hacer "nada"**
---
En su forma más sencilla, `RunnablePassthrough` recibe un input y **devuelve exactamente el mismo output**.

#### **¿Por qué usar algo tan sencillo?**
A veces en una Chain necesitas un paso que no modifique los datos, pero que sea compatible con la secuencia de Runnables. Por ejemplo, un eslabón que valide un formato o simplemente reenvíe la información a otro paso.



In [ ]:
from langchain_core.runnables import RunnablePassthrough

# Creamos un RunnablePassthrough
passthrough = RunnablePassthrough()

# Definimos un texto de ejemplo
texto_entrada = "Este texto será pasado sin cambios."

# Ejecutamos el RunnablePassthrough
resultado = passthrough.invoke(texto_entrada)

print("Texto de entrada: ", texto_entrada)
print("Resultado       : ", resultado)


Texto de entrada :  Este texto será pasado sin cambios.
Resultado        :  Este texto será pasado sin cambios.


## **Ejemplo 2: Integrar RunnablePassthrough en una cadena**
---
Este ejempplo puede parecer un poco forzada pero trata de mostrar como se integra facilmente un RunnablePassthrough en una cadena.

Pasaremos un promt a un LLM y el resultado de este pasara a traves del RunnablePassthrough a otro prompt y de ahi a otro LLM.



In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

prompt_template1 = PromptTemplate.from_template("Describe y condensa el siguinte tema en una sola frase: {tema}")
prompt_template2 = PromptTemplate.from_template("Describe y condensa la siguinete frase en una sola palabra: {frase}")

llm_gpt4o_mini = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=1)

mayusculas = RunnableLambda(lambda x: x.content.upper())

chain = prompt_template1 | llm_gpt4o_mini | RunnablePassthrough() | prompt_template2 | llm_gpt4o_mini | mayusculas

resultado=chain.invoke({"tema": "La inteligencia artificial"})

resultado




'AUTOMATIZACIÓN.'

Te has dado cuenta de la "dificultad" de conocer lo que esta pasando dentro de la cadena ? Cual es la frase generada por el primer modelo ?

Aqui podriamos hacer uso del RunnableLambda para imprimir el resultado intermedio en consola o cualquier otra tarea de debugger


In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

prompt_template1 = PromptTemplate.from_template("Describe y condensa el siguinte tema en una sola frase: {tema}")
prompt_template2 = PromptTemplate.from_template("Describe y condensa la siguinete frase en una sola palabra: {frase}")

llm_gpt4o_mini = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=1)

mayusculas = RunnableLambda(lambda x: x.content.upper())

def imprimir_log(x):
  print(x.content)
  return x  # es importante devolver lo mismo que recibimos

imprimir_log_runnable = RunnableLambda(imprimir_log)

chain = prompt_template1 | llm_gpt4o_mini | imprimir_log_runnable | RunnablePassthrough() | prompt_template2 | llm_gpt4o_mini | mayusculas

resultado=chain.invoke({"tema": "La inteligencia artificial"})

resultado

La inteligencia artificial es la simulación de procesos de inteligencia humana por parte de sistemas computacionales, que buscan realizar tareas como el aprendizaje, razonamiento y autocorrección.


'INTELIGENCIA.'

## **Ejemplo 3: Prepocesamiento y postprocesamiento de datos**
---
ara un caso de uso más realista, pensemos en un escenario donde:

1. Tenemos un texto que **antes** de ser enviado a un LLM debe sufrir cierta limpieza o transformación (por ejemplo, eliminar caracteres especiales).
2. Usamos el `RunnablePassthrough` como un punto de control que, en determinado momento, no hace nada, pero **podría** servir para inyectar validaciones o monitorear el flujo de datos.
3. Tras obtener la respuesta del LLM, aplicamos algún postprocesamiento (usando un `RunnableLambda` o un parser).

En un caso de producción podrías sustituir el passthrough por un paso de verificación, logging o incluso dejarlo como está si no necesitas modificar. La fortaleza de RunnablePassthrough es que no interfiere pero sí mantiene la estructura de un Runnable, lo cual facilita escalar el pipeline en el futuro.

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Supongamos que tenemos una función de preprocesamiento:
def limpiar_texto(texto):
    # Eliminamos caracteres no alfanuméricos (muy básico)
    import re
    texto_limpio = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ\s]', '', texto)
    return texto_limpio

# Convertimos esta función en un RunnableLambda:
preprocesador = RunnableLambda(limpiar_texto)

# El RunnablePassthrough lo usaremos como "puente"
passthrough = RunnablePassthrough()

# Para postprocesar, supongamos que queremos agregar metadata:
def agregar_metadata(respuesta):
    return {
        "respuesta": respuesta,
        "metadata": "Procesado con LLMChain y validado"
    }

postprocesador = RunnableLambda(agregar_metadata)



pipeline_limpieza = preprocesador | passthrough | postprocesador
    # Paso 1: Limpieza del texto
    # Paso 2: Passthrough (punto de control)
    # Paso 3: Postprocesamiento


# Probemos con un texto con símbolos:
texto_input = "¡Hola! ¿Qué tal? LangChain 4ever #1"

resultado_avanzado = pipeline_limpieza.invoke(texto_input)
print("Resultado avanzado:", resultado_avanzado)

Resultado avanzado: {'respuesta': 'Hola Qué tal LangChain ever ', 'metadata': 'Procesado con LLMChain y validado'}


# **2. `RunnablePassthrough.assign()`**
---
El método `.assign()` de `RunnablePassthrough` ofrece un mecanismo para **extender el estado actual de una cadena** sin modificar sus valores existentes. Su funcionamiento clave es:

1.  **Preserva** todos los datos originales del flujo
    
2.  **Añade nuevos campos** mediante transformaciones programáticas
    
3.  **Estructura automática** en diccionario para su uso en pasos posteriores


Es un atajo práctico para inyectar o complementar datos sin tener que construir manualmente un `RunnableLambda`

La entrada de `RunnablePassthrough.assign()` debe ser un diccionario

## **Ejemplo 1: Añadimos dos claves al diccionario**
---



In [ ]:
from langchain_core.runnables import RunnablePassthrough

cadena_enriquecida = RunnablePassthrough().assign(
    longitud=lambda x: len(x['input']),     # Nuevo campo calculado
    formato=lambda x: type(x['input'])      # Metadato adicional
)




# Ejecución con entrada de texto
resultado = cadena_enriquecida.invoke({"input": "LangChain"})
print(resultado)

{'input': 'LangChain', 'longitud': 9, 'formato': <class 'str'>}


Ejemplo 2: Enriquecer datos

Estás procesando textos para incluir una marca de tiempo simple

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from datetime import datetime

# Crear un objeto de RunnablePassthrough
add_timestamp = RunnablePassthrough().assign(timestamp=lambda x: datetime.now().isoformat())

# Procesar un texto
output = add_timestamp.invoke({"text": "Hola, mundo"})
print(output)

{'text': 'Hola, mundo', 'timestamp': '2025-01-21T19:45:12.244390'}


In [ ]:
from re import X
from os import X_OK
%pip install langdetect -qU

from langdetect import detect

# Crear un objeto de RunnablePassthrough
add_language_and_unique_words = RunnablePassthrough().assign(
    language=lambda x: detect(x["text"]),
    unique_words=lambda x: len(set(x["text"].split()))
)

# Procesar un texto
output = add_language_and_unique_words.invoke({
    "text": "Hola, este es un texto en español. ¡Hola de nuevo!"
})
print(output)

{'text': 'Hola, este es un texto en español. ¡Hola de nuevo!', 'language': 'es', 'unique_words': 10}


In [ ]:
from langchain_core.runnables import RunnablePassthrough
from datetime import datetime

# Función para enriquecer perfil de usuario
usuario_enriquecido = RunnablePassthrough().assign(
    edad_categoria=lambda x: (
        "Niño" if x['edad'] < 12 else
        "Adolescente" if x['edad'] < 18 else
        "Adulto" if x['edad'] < 60 else
        "Adulto Mayor"
    ),
    fecha_registro=lambda x: datetime.now().strftime("%Y-%m-%d")
)

# Caso de uso: Clasificación de usuario
resultado = usuario_enriquecido.invoke({
    "nombre": "Carlos Mendez",
    "edad": 25
})
print(resultado)

{'nombre': 'Carlos Mendez', 'edad': 25, 'edad_categoria': 'Adulto', 'fecha_registro': '2025-01-21'}


In [ ]:
%pip install textblob -qU

from langchain_core.runnables import RunnablePassthrough
import re
from textblob import TextBlob

def limpiar_texto(texto):
    # Eliminar caracteres especiales y convertir a minúsculas
    texto_limpio = re.sub(r'[^a-zA-Z\s]', '', texto.lower())
    return texto_limpio

analisis_texto = RunnablePassthrough().assign(
    texto_limpio=lambda x: limpiar_texto(x['texto']),
    longitud=lambda x: len(x['texto'].split()),
    sentimiento=lambda x: TextBlob(x['texto']).sentiment.polarity,
    sentimiento_categoria=lambda x: (
        "Positivo" if TextBlob(x['texto']).sentiment.polarity > 0.2 else
        "Negativo" if TextBlob(x['texto']).sentiment.polarity < -0.2 else
        "Neutral"
    )
)

# Caso de uso: Análisis de reseña de producto
resultado = analisis_texto.invoke({
    "texto": "This product is amazing! Highly recommended. 5 stars ⭐"})
print(resultado)

{'texto': 'This product is amazing! Highly recommended. 5 stars ⭐', 'texto_limpio': 'this product is amazing highly recommended  stars ', 'longitud': 9, 'sentimiento': 0.45500000000000007, 'sentimiento_categoria': 'Positivo'}


Desafio:

Eres un desarrollador encargado de diseñar un sistema de análisis para procesar datos de usuarios en un flujo de trabajo. Cada entrada contiene el nombre del usuario y un mensaje escrito por él. Tu tarea es implementar una solución utilizando el método `assign()` de `RunnablePassthrough` que realice las siguientes operaciones:

1.  **Determinar la longitud del mensaje del usuario:** Clasificar como:
    
    -   `"corto"` si tiene menos de 20 caracteres.
    -   `"mediano"` si tiene entre 20 y 50 caracteres.
    -   `"largo"` si tiene más de 50 caracteres.
2.  **Contar las palabras únicas del mensaje.**
    
3.  **Generar un identificador único (ID) para cada usuario:** Usa el formato `user_<número>` donde el número sea un contador incremental comenzando desde 1.



In [ ]:
contador = 0

def incrementar_contador(_:int) -> str:
    global contador
    contador += 1
    return f"user_{contador}"

datos = [
    {"usuario": "Ana", "mensaje": "Hola, ¿cómo estás?"},
    {"usuario": "Luis", "mensaje": "Este es un mensaje de prueba para verificar la longitud."},
    {"usuario": "Carla", "mensaje": "¡Wow! Esto es impresionante, realmente útil."}
]

clasificador_mensaje = RunnablePassthrough().assign(
    longitud=lambda x: (
        "corto" if len(x["mensaje"]) < 20 else
        "mediano" if 20 <= len(x["mensaje"]) <= 50 else
        "largo"
    ),
    palabras_unicas=lambda x: len(set(x["mensaje"].split())),
    id=lambda x:   (incrementar_contador(x))
)

for dato in datos:
    resultado = clasificador_mensaje.invoke(dato)
    print(resultado)

{'usuario': 'Ana', 'mensaje': 'Hola, ¿cómo estás?', 'longitud': 'corto', 'palabras_unicas': 3, 'id': 'user_1'}
{'usuario': 'Luis', 'mensaje': 'Este es un mensaje de prueba para verificar la longitud.', 'longitud': 'largo', 'palabras_unicas': 10, 'id': 'user_2'}
{'usuario': 'Carla', 'mensaje': '¡Wow! Esto es impresionante, realmente útil.', 'longitud': 'mediano', 'palabras_unicas': 6, 'id': 'user_3'}
